<a href="https://colab.research.google.com/github/satanjj10/TKB_LineBot/blob/main/Demo_%E6%94%BE%E6%B4%8BLineBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 此處不動

!pip install line-bot-sdk flask flask-ngrok google-cloud-logging

from flask import Flask, request, abort, jsonify
import json
from flask_ngrok import run_with_ngrok

from linebot import (
    LineBotApi, WebhookHandler
)

from linebot.exceptions import (
    InvalidSignatureError
)

from linebot.models import *

from linebot.models.template import (
    ButtonsTemplate,CarouselTemplate,ConfirmTemplate,ImageCarouselTemplate    
)

from linebot.models.template import *

from linebot.models import (
    MessageEvent, TextMessage, PostbackEvent
)
import requests
import urllib.request
import os

from linebot.models.events import (
    FollowEvent
)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("LG+AB1pEGC4U6qLFXYsbYEADiFYYGjIzSSiPBbwJrhMsEriGRgT1I23t2AIWGkZpX7iQcuT2mFapVSetiKvLuUAx8LuDdTrl/acOcqe7y2U+UEfPfFMgbmQOJUJ5sZA43vJES5nSTfE1qqjTu/cXnQdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("c7b8231cbb1aec69d8e647b389982f8d")

In [3]:
@app.route("/callback", methods=['POST'])

def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)
    print(body)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

In [17]:
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)

    with open("users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\n')

        line_bot_api.reply_message(
        event.reply_token,
        [
            TextSendMessage('感謝您加入放洋留遊學！\n\n我是XXX(吉祥物)，身為你的專屬客服，讓我來為你解惑吧!\n\n這裡會由我公告留遊學最新通知、活動、優惠，也可以和我一對一對話，還可以點選下方打字機圖示，了解更多喔~'),
            TextSendMessage('花個10秒鐘，完成以下選單，即可獲得免費資深專家約談喔!\n\n先讓我認識你吧~'),
            LocationSendMessage(
                                title='服務據點',
                                address='台北市忠孝西路一段 102 號 3F',
                                latitude=25.04717,
                                longitude=121.51264),
            TemplateSendMessage(
                                alt_text='ImageCarousel template',
                                template=ImageCarouselTemplate(
                                                                columns=[
                                                                          ImageCarouselColumn(
                                                                          image_url='https://cdn.pixabay.com/photo/2021/11/02/18/58/airline-6763903_1280.png',
                                                                          action=MessageAction(
                                                                          label='我想去美國',
                                                                          text='我想去美國')
                                                              ),
                                                                          ImageCarouselColumn(
                                                                          image_url='https://cdn.pixabay.com/photo/2021/11/02/18/58/airline-6763903_1280.png',
                                                                          action=MessageAction(
                                                                          label='我想去英國',
                                                                          text='我想去英國')
                                                              ),
                                                                          ImageCarouselColumn(
                                                                          image_url='https://cdn.pixabay.com/photo/2021/11/02/18/58/airline-6763903_1280.png',
                                                                          action=MessageAction(
                                                                          label='我想去澳洲',
                                                                          text='我想去澳洲')
                                                              ),
                                                                          ImageCarouselColumn(
                                                                          image_url='https://cdn.pixabay.com/photo/2021/11/02/18/58/airline-6763903_1280.png',
                                                                          action=MessageAction(
                                                                          label='我想去加拿大',
                                                                          text='我想去加拿大')
                                                              ),]

                                ))
         ]
    )

In [27]:
# 為按鈕命名
p1 = TemplateSendMessage(
                         alt_text='想要留學還是短期遊學?',
                         template=ButtonsTemplate(
                                                  title='想要留學還是短期遊學?',
                                                  actions=[
                                                  MessageAction(
                                                  label='留學',
                                                  text='我想留學'), 
                                                  MessageAction(
                                                  label='短期遊學',
                                                  text='我想遊學')]))


In [24]:
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    if (event.message.text.find('我想去美國')!= -1):       
        line_bot_api.reply_message(event.reply_token, [
                                                  TemplateSendMessage(
                                                                      alt_text='想要留學還是短期遊學?',
                                                                      template=ButtonsTemplate(
                                                                                                title='想要留學還是短期遊學?',
                                                                                                text='請點選以下按鈕',
                                                                                                actions=[
                                                                                                     MessageAction(
                                                                                                        label='留學',
                                                                                                        text='我想留學'), 
                                                                                                     MessageAction(
                                                                                                        label='短期遊學',
                                                                                                        text='我想遊學'),                                                                                              
                                                                                                 ]))])
    elif (event.message.text.find('我想留學')!= -1):       
        line_bot_api.reply_message(event.reply_token, [
                                                  TemplateSendMessage(
                                                                      alt_text='我的英文程度',
                                                                      template=ButtonsTemplate(
                                                                                                title='我的英文程度',
                                                                                                text='請點選以下按鈕',
                                                                                                actions=[                                                                                            
                                                                                                    MessageAction(
                                                                                                        label='高',
                                                                                                        text='我的英文頂呱呱'),
                                                                                                    MessageAction(
                                                                                                        label='中',
                                                                                                        text='我的英文還可以'),
                                                                                                    MessageAction(
                                                                                                        label='低',
                                                                                                        text='我的英文還需加強'),                                                                                              
                                                                                                 ]))])
    elif (event.message.text.find('我的英文頂呱呱')!= -1):       
        line_bot_api.reply_message(event.reply_token, [
                                                  TemplateSendMessage(
                                                                      alt_text='是否確認進入真人諮詢服務?',
                                                                      template=ConfirmTemplate(
                                                                                                text='是否確認進入真人諮詢服務?',
                                                                                                actions=[
                                                                                                    MessageAction(
                                                                                                        label='是',
                                                                                                        text='呼叫小編!!!'),
                                                                                                    MessageAction(
                                                                                                        label='否',
                                                                                                        text='請點選下方打字機圖片點選查看更多資訊')
                                                                                                ]))])

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://98fe-34-80-130-102.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [09/Dec/2022 08:08:43] "POST /callback HTTP/1.1" 200 -


{"destination":"U27545d4bec88860a1fb4b67195c22408","events":[]}


...

{"destination":"U27545d4bec88860a1fb4b67195c22408","events":[{"type":"message","message":{"type":"text","id":"17264583786454","text":"我的英文頂呱呱"},"webhookEventId":"01GKTYWQQCFTC6WDKG61RK0NTF","deliveryContext":{"isRedelivery":false},"timestamp":1670573350481,"source":{"type":"user","userId":"Uf57398b2870127a0567cc1fc518d93e0"},"replyToken":"5a53cfe4d9b64d0c8872de1d43973497","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [09/Dec/2022 08:09:15] "POST /callback HTTP/1.1" 200 -


{"destination":"U27545d4bec88860a1fb4b67195c22408","events":[{"type":"message","message":{"type":"text","id":"17264584132892","text":"呼叫小編!!!"},"webhookEventId":"01GKTYWVG11BA6SJFK94EG4XNQ","deliveryContext":{"isRedelivery":false},"timestamp":1670573354389,"source":{"type":"user","userId":"Uf57398b2870127a0567cc1fc518d93e0"},"replyToken":"d1c8fa3eb4244fa5b9420dc974882581","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [09/Dec/2022 08:10:15] "POST /callback HTTP/1.1" 200 -


{"destination":"U27545d4bec88860a1fb4b67195c22408","events":[{"type":"message","message":{"type":"text","id":"17264589244590","text":"Yes"},"webhookEventId":"01GKTYYPA30J60Q4GPYBQVCZTN","deliveryContext":{"isRedelivery":false},"timestamp":1670573414487,"source":{"type":"user","userId":"Uf57398b2870127a0567cc1fc518d93e0"},"replyToken":"f1dc10972ff64f4384a253a2030b7430","mode":"active"}]}
